# Little Lemon Analysis and Sales Report

In [3]:
# Task 1:

from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig = {
    "database":"little_lemon_db",
    "user" : "root",
    "password" : "********"
}

try:
    pool = MySQLConnectionPool(pool_name = "pool_b",
                           pool_size = 2, #default is 5
                           **dbconfig)
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)

except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

The connection pool is created with a name:  pool_b
The pool size is: 2


## Task 2
Three guests are trying to book dinner slots simultaneously. 

Let's get the connections from `pool_b` and insert the following data in the Bookings table:

Guest 1:

* Table Number: 8

* First Name: Anees

* Last Name: Java

* Booking Time: 18:00

* EmployeeID: 6


Guest 2:

* Table Number: 5

* First Name: Bald

* Last Name: Vin

* Booking Time: 19:00

* EmployeeID: 6

Guest 3:

* Table Number: 12

* First Name: Jay

* Last Name: Kon

* Booking Time: 19:30 

* EmployeeID: 6


**NB:** The pool size is 2, we have to add a new connection for Guest 3

In [4]:
# Let's Connect the first guest.
connection1 = pool.get_connection()
print("Connection1 is created!")
cursor1 = connection1.cursor()
booking1="""INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(8,'Anees','Java','18:00:00',6);"""

cursor1.execute(booking1)
connection1.commit()
print("""A new booking is added in the "Bookings" table.""")

# Let's Connect the second guest.
connection2 = pool.get_connection()
print("Connection2 is created!")
cursor2=connection2.cursor()
booking2="""INSERT INTO Bookings
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(5, 'Bald','Vin','19:00:00',6);"""
cursor2.execute(booking2)
connection2.commit()
print("""A new booking is added in the "Bookings" table.""")

# Adding a new connection to connect the third user.

import mysql.connector as connector
try:
    connection3 = pool.get_connection()
    print("The guest is connected")
except:
    print("Adding new connection in the pool.")
        
    # Create a connection
    connection=connector.connect(user="root",password= "Ibraheem16@mysql")
    # Add the connection into the pool
    pool.add_connection(cnx=connection)
    print("A new connection is added in the pool.\n")
        
    connection3 = pool.get_connection()
    print("'connection3' is added in the pool.")
    
# Connect the third guest 
cursor3=connection3.cursor()
booking3="""INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(12, 'Jay','Kon','19:30:00',6);"""
cursor3.execute(booking3)
connection3.commit()
print("""A new booking is added in the "Bookings" table.""")

# We can only return two connections back to the pool as the pool_size=2.
# Let's Close all  connections and use try-except to print the pool error if the pool is already full.

from mysql.connector import Error
for connection in [connection1,connection2, connection3]:
    try:
        connection.close()
        print("Connection is returned to the pool")
    except Error as er:
        print("\nConnection can't be returned to the pool")
        print("Error message:", er.msg)


Connection1 is created!
A new booking is added in the "Bookings" table.
Connection2 is created!
A new booking is added in the "Bookings" table.
Adding new connection in the pool.
A new connection is added in the pool.

'connection3' is added in the pool.
A new booking is added in the "Bookings" table.
Connection is returned to the pool
Connection is returned to the pool

Connection can't be returned to the pool
Error message: Failed adding connection; queue is full


## Task 3

Let's create a report containing the following information:

1. The name and EmployeeID of the Little Lemon manager.

2. The name and role of the employee who receives the highest salary.

3. The number of guests booked between 18:00 and 20:00.

4. The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [5]:
# Let's Get a connection from pool_a and create a cursor object to communicate with the database. 
print("Getting a connection from the pool.")
connection = pool.get_connection()
print("""The object "connection" is created with a connection link from the pool_a""")
print("""Creating a cursor object to communicate with the database.""")
cursor=connection.cursor()
print("""The cursor object "cursor" is created.""")

# The name and EmployeeID of the Little Lemon manager.

print("The name and EmployeeID of the Little Lemon manager:")
cursor.execute("""SELECT 
Name, EmployeeID 
FROM Employees
WHERE Role = 'Manager'""")

results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)


# The name and role of the employee who receives the highest salary
print("The name and role of the employee who receives the highest salary:")
cursor.execute("""SELECT 
Name, EmployeeID 
FROM Employees ORDER BY 
Annual_Salary DESC LIMIT 1""")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)


# The number of guests booked between 18:00:00 and 20:00:00.
print("The number of guests booked between 18:00:00 and 20:00:00:")
cursor.execute("""SELECT 
COUNT(BookingID) n_booking_between_18_20_hrs
FROM Bookings 
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';""")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

# Full name and the BookingId of each guest  waiting to be seated  
# with the receptionist in sorted order with respect to their BookingSlot.

cursor.execute("""
SELECT

Bookings.BookingID AS ID,
CONCAT(GuestFirstName,' ',GuestLastName) AS GuestName,

Role AS Employee

FROM Bookings
LEFT JOIN
Employees
ON Employees.EmployeeID=Bookings.EmployeeID
WHERE Employees.Role = "Receptionist"
ORDER BY BookingSlot DESC;

""")
print("The following guests are waiting to be seated:")
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)


Getting a connection from the pool.
The object "connection" is created with a connection link from the pool_a
Creating a cursor object to communicate with the database.
The cursor object "cursor" is created.
The name and EmployeeID of the Little Lemon manager:
('Name', 'EmployeeID')
('Mario Gollini', 1)
The name and role of the employee who receives the highest salary:
('Name', 'EmployeeID')
('Mario Gollini', 1)
The number of guests booked between 18:00:00 and 20:00:00:
('n_booking_between_18_20_hrs',)
(14,)
The following guests are waiting to be seated:
('ID', 'GuestName', 'Employee')
(9, 'Jay Kon', 'Receptionist')
(13, 'Jay Kon', 'Receptionist')
(16, 'Jay Kon', 'Receptionist')
(8, 'Bald Vin', 'Receptionist')
(12, 'Bald Vin', 'Receptionist')
(15, 'Bald Vin', 'Receptionist')
(7, 'Anees Java', 'Receptionist')
(10, 'Anees Java', 'Receptionist')
(11, 'Anees Java', 'Receptionist')
(14, 'Anees Java', 'Receptionist')


## Task 4

We need a stored procedure called BasicSalesReport that returns a the sales statistics including: 

* Total sales
* Average sale
* Minimum bill paid
* Maximum bill paid

In [6]:
# Create a stored procedure named BasicSalesReport. 
cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport;")

stored_procedure_query="""
CREATE PROCEDURE BasicSalesReport()

BEGIN
    SELECT 
        SUM(BillAmount) AS Total_Sale,
        AVG(BillAmount) AS Average_Sale,
        MIN(BillAmount) AS Min_Bill_Paid,
        MAX(BillAmount) AS Max_Bill_Paid
    FROM Orders;
    END
"""

# Execute the query
cursor.execute(stored_procedure_query)

#********************************************#

# Call the stored procedure with its name
cursor.callproc("BasicSalesReport")

# Retrieve records in "dataset"
results = next(cursor.stored_results())
results = results.fetchall()

# Retrieve column names using list comprehension in a for loop 
for column_id in cursor.stored_results():
    cols = [column[0] for column in column_id.description]

    
print("Today's sales report:")
for result in results:
    print("\t",cols[0],":",result[0])
    print("\t",cols[1],":",result[1])
    print("\t",cols[2],":",result[2])
    print("\t",cols[3],":",result[3])


Today's sales report:
	 Total_Sale : 243
	 Average_Sale : 48.6000
	 Min_Bill_Paid : 37
	 Max_Bill_Paid : 86


## Task 5

Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs that the orders are due. 

In [7]:
connection = pool.get_connection()
cursor=connection.cursor(buffered=True)

sql_query="""SELECT 
Bookings.BookingSlot,
CONCAT(Bookings.GuestFirstName," ",Bookings.GuestLastName) AS Guest_Name,
Employees.Name AS Emp_Name,
Employees.Role AS Emp_Role
FROM Bookings 
INNER JOIN 
Employees ON Bookings.EmployeeID=Employees.EmployeeID
ORDER BY Bookings.BookingSlot ASC;"""
# The sql query Combines the data from the Bookings and the Employee tables, sorts the retrieved records in ascending order and then displays the information of the first three guests. 

cursor.execute(sql_query)
results=cursor.fetchmany(size=3)
#print(cursor.column_names)
# Let's print the BookingSlot, Guest name and Assigned employee in a more user friendly format
for result in results:
    print("\nBookingSlot",result[0])
    print("\tGuest_name:",result[1])
    print("\tAssigned to:",result[2],"[{}]".format(result[3]))
    
connection.close()
print("Connection closed")


BookingSlot 15:00:00
	Guest_name: Vanessa McCarthy
	Assigned to: Giorgos Dioudis [Head Chef]

BookingSlot 17:30:00
	Guest_name: Marcos Romero
	Assigned to: Fatma Kaya [Assistant Chef]

BookingSlot 18:00:00
	Guest_name: Anees Java
	Assigned to: John Millar [Receptionist]
Connection closed
